In [1]:
import pandas as pd
import html
#nltk.download() #only for the first time running it
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
import json
import regex as re
from textblob import TextBlob
import numpy as np
from wordcloud import WordCloud, STOPWORDS

In [ ]:
#create dataframe on media posts USE FOR ALL DATA - Don't run now
def createDataFrame():  
    with open("datain/nft_search_tweets_sample.jsonl", encoding='utf8') as f:
        for line in f:
            row = json.loads(line)
            try:
                if row["lang"] == 'en' and len(row.get('referenced_tweets', [])) == 0 and len(row.get('entities', ['hashtags'])) != 0:
                    yield (row['id'],
                            row['text'],
                            row['entities']['hashtags'],
                            row['public_metrics']['retweet_count'],
                            row['public_metrics']['quote_count'],
                            row['public_metrics']['reply_count'],
                            row['public_metrics']['like_count'],
                            row['attachments']['media_keys'])
            except KeyError:
                pass

#create csv
df = pd.DataFrame(createDataFrame())
df.columns =['id', 'text', 'hashtags', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'media_keys']
df.to_json('datain/nft_tweets.jsonl', orient='records', index=True, lines= True)
output = pd.read_json("datain/nft_tweets.jsonl", lines = True)
output['total'] = output[['retweet_count', 'quote_count','reply_count', 'like_count']].sum(axis=1)
#sort by highest total
output = output.sort_values(by = 'total', ascending = False)
#top 100
output = output.head(100)
output.to_json('datain/nft_top_100_tweets.jsonl', orient='records', index=True, lines= True)

In [ ]:
#import the data
data = pd.read_json("datain/nft_top_100_tweets.jsonl", lines=True)

In [32]:
sample_data = pd.read_json("datain/nft_search_tweets_sample.jsonl", lines=True)

In [35]:
#prep stop words
stop_words = stopwords.words('english')
stop_words.append('rt')
stop_words.append('nft')
# stop_words.append('#nft')

#function for cleaning a tweet (remove mentions, hashtags, links, html entities, stop words. And make sure it's only letters)
def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        tweet = str.lower(tweet)
        # tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)", " ", tweet).split()) # remove mentions and hashtags
        # tweet = ' '.join(re.sub("(@[A-Za-z0-9_]+)", " ", tweet).split())
        tweet = re.sub("(http\S+|http)", "", tweet, flags=re.MULTILINE) # remove links
        tweet = re.sub('\&\w+', "", tweet) # remove html entities
        tweet = re.sub('[^a-zA-Z# ]+', ' ', tweet) # make sure tweet is only letters
        # stem & remove stop words
        # tweet = ' '.join([PorterStemmer().stem(word=word) for word in tweet.split() if word not in stop_words])
        tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
        return tweet

#clean data
for i in sample_data.index:
    text = sample_data["text"][i]
    cleaned_text = clean_tweet(text)
    cleaned_text = html.unescape(cleaned_text)
    sample_data["text"][i] = cleaned_text

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
sample_data.to_json('datain/nft_search_tweets_sample_cleaned.jsonl', orient='records', index=True, lines= True)

In [38]:
tweet_text = sample_data['text'].str.cat(sep=' ')
tokens = nltk.word_tokenize(tweet_text)
most_common = pd.DataFrame(nltk.ngrams(tokens, 1)).value_counts().to_frame()
# terms_count = term_data['text'].value_counts().to_dict()
# terms_count = pd.DataFrame.from_dict(terms_count, orient='index')

# terms_count.to_html("dataout/terms_count.html")

most_common.to_html('dataout/term_freq.html')

In [4]:
#nft_top_100_sentiment <- Has interactions and sentiment totals
positivity = ''
output = pd.read_json("datain/nft_top_100_cleaned_interactions.jsonl", lines = True)
def getSentiment():
    for i in output.index:
        row = TextBlob(output.iloc[i]['text'])
        if row.sentiment.polarity >= 0.7:
                positivity = 'mostly_positive'
        elif row.sentiment.polarity <= -0.7:
            positivity = 'mostly_negative'
        elif row.sentiment.polarity > -0.7 and row.sentiment.polarity < -0.4:
            positivity = 'negative'
        elif row.sentiment.polarity > 0.4 and row.sentiment.polarity < 0.7:
                positivity = 'positive'
        else:
            positivity = 'nuetral'
        yield  row.sentiment.polarity, row.sentiment.subjectivity, positivity
        
df = pd.DataFrame(getSentiment())
df.columns =['polarity', 'subjectivity', 'positivity']
output['polarity'] = df['polarity']
output['subjectivity'] = df['subjectivity']
output['positivity'] = df['positivity']
output.to_json('datain/nft_top_100_cleaned_interactions_sentiment.jsonl', orient='records', index=True, lines= True)

In [ ]:
# Word cloud of top 100 NFT's
top_100_word_cloud = pd.read_json("datain/nft_top_100_cleaned_interactions_sentiment.jsonl", lines=True)
# Generate a word cloud image
stopwords = set(STOPWORDS)
stopwords.update(("t", "co", "https", "t", "amp", "RT"))
wordcloud = WordCloud(stopwords=stopwords,background_color='white', max_words=1000,contour_color='#023075',contour_width=3,colormap='rainbow').generate(' '.join(top_100_word_cloud['text']))
# create image as cloud
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
# store to file
plt.savefig("cloud.png", format="png")
plt.show()

In [8]:
# Comment sentiment
#### Read from nft_search_tweets_sample - need replied to with text and author
#### Original Poster
def createOriginalDataFrame():  
    with open("datain/nft_search_tweets_sample.jsonl", encoding='utf8') as f:
        for line in f:
            row = json.loads(line)
            try:
                if row["lang"] == 'en'and len(row.get('referenced_tweets', [])) == 0:
                    yield (row['id'],
                            row['text'],
                            row['author_id'],
                            row['public_metrics']['retweet_count'],
                            row['public_metrics']['quote_count'],
                            row['public_metrics']['reply_count'],
                            row['public_metrics']['like_count'],
                            row['attachments']['media_keys']
                    )
            except KeyError:
                pass

def createRepliesDataFrame():  
    with open("datain/nft_search_tweets_sample.jsonl", encoding='utf8') as f:
        for line in f:
            row = json.loads(line)
            try:
                if row["lang"] == 'en'and len(row.get('referenced_tweets', [])) != 0:
                    yield (row['id'],
                            row['text'],
                            row['author_id'],
                            row['in_reply_to_user_id'],
                            row['public_metrics']['retweet_count'],
                            row['public_metrics']['quote_count'],
                            row['public_metrics']['reply_count'],
                            row['public_metrics']['like_count'],
                            row['attachments']['media_keys']
                    )
            except KeyError:
                pass
#create csv
original_data = pd.DataFrame(createOriginalDataFrame())
original_data.columns =['id', 'text', 'in_reply_to_user_id', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'media_keys']
original_data.to_json('datain/nft_original_tweets.jsonl', orient='records', index=True, lines= True)
original_data['total'] = original_data[['retweet_count', 'quote_count','reply_count', 'like_count']].sum(axis=1)

replies_data = pd.DataFrame(createRepliesDataFrame())
replies_data.columns =['id', 'text', 'reply_author_id', 'in_reply_to_user_id', 'retweet_count', 'quote_count', 'reply_count', 'like_count', 'media_keys']
replies_data.to_json('datain/nft_replies_tweets.jsonl', orient='records', index=True, lines= True)
replies_data['total'] = replies_data[['retweet_count', 'quote_count','reply_count', 'like_count']].sum(axis=1)

In [9]:
original_data.reindex()
replies_data.reindex()

all_data = pd.merge(original_data, replies_data, on='in_reply_to_user_id').dropna()

# replies_data['matched_id'] = pd.Series((original_data.original_author_id.isin(replies_data.reply_author_id)))

In [10]:
def getSentiment():
    for i in all_data.index:
        row = TextBlob(all_data.iloc[i]['text_y'])
        if row.sentiment.polarity >= 0.7:
                positivity = 'mostly_positive'
        elif row.sentiment.polarity <= -0.7:
            positivity = 'mostly_negative'
        elif row.sentiment.polarity > -0.7 and row.sentiment.polarity < -0.4:
            positivity = 'negative'
        elif row.sentiment.polarity > 0.4 and row.sentiment.polarity < 0.7:
                positivity = 'positive'
        else:
            positivity = 'nuetral'
        yield  row.sentiment.polarity, row.sentiment.subjectivity, positivity
    # with open("datain/cleaned.txt", encoding='utf8') as f:
    #     for line in f:
    #         row = TextBlob(line)
    #         if row.sentiment.polarity >= 0.7:
    #             positivity = 'mostly_positive'
    #         elif row.sentiment.polarity <= -0.7:
    #             positivity = 'mostly_negative'
    #         elif row.sentiment.polarity > -0.7 and row.sentiment.polarity < -0.4:
    #             positivity = 'negative'
    #         elif row.sentiment.polarity > 0.4 and row.sentiment.polarity < 0.7:
    #              positivity = 'positive'
    #         else:
    #             positivity = 'nuetral'
    #         yield line, row.sentiment.polarity, row.sentiment.subjectivity, positivity

df = pd.DataFrame(getSentiment())
df.columns =['polarity', 'subjectivity', 'positivity']
all_data['polarity'] = df['polarity']
all_data['subjectivity'] = df['subjectivity']
all_data['positivity'] = df['positivity']
all_data.to_json('datain/matched_author_replied_sentiment.jsonl', orient='records', index=True, lines= True)

In [ ]:
def createTagDataFrame():  
    with open("datain/nft_search_tweets_sample.jsonl", encoding='utf8') as f:
        for line in f:
            row = json.loads(line)
            try:
                if row["lang"] == 'en' and len(row.get('entities', ['hashtags'])) != 0:
                    yield (row['id'],
                            row['text'],
                            row['entities']['hashtags'])
            except KeyError:
                pass
#create csv
tag_data = pd.DataFrame(createTagDataFrame())
tag_data.columns =['id', 'text', 'hashtags']
new_df = pd.concat([pd.DataFrame(pd.json_normalize(x)) for x in tag_data['hashtags']],ignore_index=True)

tags_count = new_df['tag'].value_counts().to_dict()
counts = pd.DataFrame.from_dict(tags_count, orient='index')


In [ ]:
tags_count.to_html("dataout/tags_count.html")


In [11]:
interactions_sentiment_df = pd.read_json('datain/matched_author_replied_sentiment.jsonl', lines=True)